### 环境准备

In [3]:
import pandas as pd
import requests
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import re
headers ={ 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}

### 项目总表

In [5]:
url = 'http://117.39.29.75:8085/pricePublic/house/public/index?id=8a901c286aa0405a016b50713e4701ad&page=1&size=100'
#url = 'http://117.39.29.75:8085/pricePublic/house/public/index'
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'lxml')

data_list = []
name_list = []
buildNo_list = []
for idx, tr in enumerate(soup.find_all('tbody')):
    tds = tr.find_all('td')
    for i in range(0,len(tds),5):
        data_list.append({
            '1项目名称': str(tds[i].contents[0])[68:-4],
            '2所属批次': tds[i+1].contents[0],
            '3公示时间': tds[i+2].contents[0],
            '4申报楼栋': tds[i+3].contents[0],
            '5总层数': tds[i+4].contents[0]
        })
        h = str(tds[i].contents[0])[18:50]
        name_list.append(h)
        k = tds[i+3].contents[0]
        buildNo_list.append(k)
data_list1 = pd.DataFrame(data_list)
data_list1[['项目名称','所属批次','公示时间','申报楼栋','总层数']] = data_list1[['1项目名称','2所属批次','3公示时间','4申报楼栋','5总层数']]
data_list1 = data_list1.iloc[:,5:]
data_list1.to_excel('C:\\Users\\l\\Desktop\\项目总表.xlsx',encoding = "gbk")

### 明细数据

In [7]:
def get_tabledata(ID,NO):
    params = {
                'id': ID, #参数一
                'page': '1',
                'size': '10000',
                'sort': '',
                'buildNo': NO,#参数二
                }
    url = 'http://117.39.29.75:8085/pricePublic/house/public/price?' + urlencode(params)
    #print(url)
    response = requests.get(url, headers=headers)
    pat = '<td>(.*?)</td>'
    pat1 = '<span style="display:inline-block;">(.*?)</span></td>'
    pat2 = '<span>(.*?)</span>'
    rst = re.compile(pat).findall(response.text)
    rst1 = re.compile(pat1).findall(response.text)
    rst2 = re.compile(pat2).findall(response.text)
    str = rst2[0]
    result = re.search('(.*?)号楼 一房一价信息,层数：(\d+)', str)
    lz = result.group(1)
    lc = result.group(2)

    table_head = {'A':['项目名称','项目地址','楼幢信息','楼层信息','序号'],
            'B':[rst1[0],rst1[2],lz,lc,'房间号'],
            'C':['装修标准','','','信息核查','建筑面积'],
            'D':[rst1[1],'','',rst2[0],'单价'],
            'E':['','','','','房屋总价'],
    }
    table_head = pd.DataFrame(table_head)
    #print(table_head)
    del rst[0]
    data_list = []
    for i in range(0,len(rst),5):
        data_list.append({
            '1序号': rst[i],
            '2房间号': rst[i+1],
            '3建筑面积': rst[i+2],
            '4房屋均价': rst[i+3],
            '5房屋总价': rst[i+4]
        })
    data_list = pd.DataFrame(data_list)
    data_list[['A','B','C','D','E']] = data_list[['1序号','2房间号','3建筑面积','4房屋均价','5房屋总价']]
    data_list[['A','C','D','E']] = data_list[['A','C','D','E']].astype(float)
    data_list = data_list.iloc[:,5:]
    data_table = pd.concat([table_head,data_list],axis=0,ignore_index=True)#纵向合并，不考虑原来的index
    data_table.to_excel("C:\\Users\\l\\Desktop\\" + rst1[0] + lz + '#.xlsx',encoding = "gbk")
    #print(data_list)

### 保存数据

In [8]:
i = 0
while i < len(buildNo_list):
    ID = name_list[i]
    str = buildNo_list[i]
    m = str.replace("、",",")
    m = m.replace("#","")
    m = m.split(',')
    for NO in m:
        get_tabledata(ID,NO)
    i = i + 1